<a href="https://colab.research.google.com/github/TeraneAgaeva/Data-Analysis-course-/blob/main/%22hw_6_4_%D0%90%D0%BD%D0%B0%D0%BB%D1%96%D0%B7_%D0%90%D0%92_%D1%82%D0%B5%D1%81%D1%82%D1%83_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Аналіз A/B тесту

Маємо проаналізувати дані A/B тесту в популярній [грі Cookie Cats](https://www.facebook.com/cookiecatsgame). Це класична гра-головоломка в стилі «з’єднай три», де гравець повинен з’єднати плитки одного кольору, щоб очистити дошку та виграти рівень. На дошці також зображені співаючі котики :)

Під час проходження гри гравці стикаються з воротами, які змушують їх чекати деякий час, перш ніж вони зможуть прогресувати або зробити покупку в додатку. У цьому блоці завдань ми проаналізуємо результати A/B тесту, коли перші ворота в Cookie Cats було переміщено з рівня 30 на рівень 40. Зокрема, ми проаналізуємо вплив на утримання (retention) гравців. Тобто хочемо зрозуміти чи переміщення воріт на 10 рівнів пізніше якимось чином вплинуло на те, що користувачі перестають грати в гру раніше чи пізніше з точки зору кількості їх днів з моменту встановлення гри.

Будемо працювати з даними з файлу `cookie_cats.csv`. Змінні в даних наступні:

- userid - унікальний номер, який ідентифікує кожного гравця.
- version - чи потрапив гравець в контрольну групу (gate_30 - ворота на 30 рівні) чи тестову групу (gate_40 - ворота на 40 рівні).
- sum_gamerounds - кількість ігрових раундів, зіграних гравцем протягом першого тижня після встановлення
- retention_1 - чи через 1 день після встановлення гравець повернувся і почав грати?
- retention_7 - чи через 7 днів після встановлення гравець повернувся і почав грати?

Коли гравець встановлював гру, його випадковим чином призначали до групи gate_30 або gate_40.

1. Зчитайте дані АВ тесту у змінну `df` та виведіть середнє значення показника показник `retention_7` (утримання на 7 день) по версіям гри. Сформулюйте гіпотезу: яка версія дає краще утримання через 7 днів після встановлення гри?

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.stats.api as sms
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from math import ceil

%matplotlib inline

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!ls drive/MyDrive/cookie_cats.csv

drive/MyDrive/cookie_cats.csv


In [4]:
data_path = 'drive/MyDrive/cookie_cats.csv'
df = pd.read_csv(data_path)

In [5]:
df.shape

(90189, 5)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90189 entries, 0 to 90188
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userid          90189 non-null  int64 
 1   version         90189 non-null  object
 2   sum_gamerounds  90189 non-null  int64 
 3   retention_1     90189 non-null  bool  
 4   retention_7     90189 non-null  bool  
dtypes: bool(2), int64(2), object(1)
memory usage: 2.2+ MB


In [7]:
df

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
2,377,gate_40,165,True,False
3,483,gate_40,1,False,False
4,488,gate_40,179,True,True
...,...,...,...,...,...
90184,9999441,gate_40,97,True,False
90185,9999479,gate_40,30,False,False
90186,9999710,gate_30,28,True,False
90187,9999768,gate_40,51,True,False


Перш ніж ми продовжимо і проаналізуємо дані для отримання нашої підмножини, давайте упевнимося, що немає користувачів, які були обрані кілька разів.

In [8]:
user_counts = df['userid'].value_counts(ascending=False)
multi_users = user_counts[user_counts > 1].count()

print(f'Є {multi_users} користувачів, які зустрічаються кілька разів у наборі даних.')

Є 0 користувачів, які зустрічаються кілька разів у наборі даних.


In [9]:
# ДОдамо поле 'Count' та заповним значенням 1, адже знаємо з попереднього аналізу, що записи у df унікальні
df['Count'] = 1

In [10]:
# Групуємо по полям 'version' и 'retention_7' та рахуємо кількість користувачів по підгруппам
grouped_data = df.groupby(['version', 'retention_7']).size().reset_index(name='count')


print(grouped_data)

   version  retention_7  count
0  gate_30        False  36198
1  gate_30         True   8502
2  gate_40        False  37210
3  gate_40         True   8279


In [11]:
# Групуємо по полям 'version' и 'retention_7' та рахуємо кількість користувачів по підгруппам
grouped_data1 = grouped_data.groupby(['version']).sum()


print(grouped_data1)

         retention_7  count
version                    
gate_30            1  44700
gate_40            1  45489


In [12]:
36198+8502

44700

In [13]:
alpha = 0.05


In [14]:
# Групування даних по 'version' та розрахунок значення для 'retention_7'
grouped = df.groupby('version')['retention_7'].mean().reset_index()

# Перейменуємо поле retention_7 на mean_retention_7
grouped = grouped.rename(columns={'retention_7': 'mean_retention_7'})

print(grouped)

   version  mean_retention_7
0  gate_30          0.190201
1  gate_40          0.182000


In [15]:
# Перевірка середнього

8502/(36198+8502)

0.19020134228187918

Почему лучше использовать **.sample()**:
Случайность: **.sample()** позволяет выбрать случайные строки из данных, что помогает избежать систематического смещения. Это особенно важно, если ваш набор данных содержит тенденции или аномалии, которые могут повлиять на результаты теста.

Контроль размера выборки: Вы можете точно определить размер выборки, что важно для статистических анализов. Если данные не равномерно распределены, **.sample()** позволяет вам настроить размер выборки для каждой группы.

Репрезентативность: Случайное подмножество данных помогает обеспечить, что ваша выборка является репрезентативной для всей популяции, что делает результаты теста более надежными и применимыми.

In [17]:
# Формуємо вибірки ВАР 1

"""робочий, але не зовсім підходящий варіант. В этом коде
вы создаете две Series (group_30 и group_40), которые содержат только значения в столбце 'count'
для каждой из групп. Эта часть кода просто извлекает данные для двух групп, но не создает случайные выборки."""

group_30 = grouped_data[grouped_data['version'] == 'gate_30']['count']
group_40 = grouped_data[grouped_data['version'] == 'gate_40']['count']

print(group_30, group_40)

0    36198
1     8502
Name: count, dtype: int64 2    37210
3     8279
Name: count, dtype: int64


In [20]:
# Формуємо вибірки ВАР 2 з використанням .sample() для вибору випадкових значень. Число 22 - рандомне значення

gate_30_sample1 = df[df['version'] == 'gate_30'].sample(n = count_gate_30, random_state=22)
gate_40_sample1 = df[df['version'] == 'gate_40'].sample(n = count_gate_40, random_state=22)

print(gate_30_sample1, gate_40_sample1)


        userid  version  sum_gamerounds  retention_1  retention_7  Count
68036  7540471  gate_30              45         True        False      1
32277  3589138  gate_30              21         True        False      1
28511  3177668  gate_30              14         True        False      1
19176  2133884  gate_30              26        False        False      1
4432    492763  gate_30              39         True         True      1
...        ...      ...             ...          ...          ...    ...
46326  5131516  gate_30              11         True        False      1
50189  5563895  gate_30              35        False        False      1
63753  7060016  gate_30               8        False        False      1
59362  6573010  gate_30               1        False        False      1
88576  9820753  gate_30              39         True        False      1

[44700 rows x 6 columns]         userid  version  sum_gamerounds  retention_1  retention_7  Count
86249  9554112  gate_40  

In [40]:
# Число користувачів, що повернулись через 7 днів у кожній групі
retention_7_gate_30 = df[df['version'] == 'gate_30']['retention_7'].sum()
retention_7_gate_40 = df[df['version'] == 'gate_40']['retention_7'].sum()

print(retention_7_gate_30, retention_7_gate_40)

8502 8279


In [41]:
# підрахунок суми рядків

# вар 1
count_gate_30 = df[df['version'] == 'gate_30'].shape[0]
count_gate_40 = df[df['version'] == 'gate_40'].shape[0]


print(count_gate_30, count_gate_40)

44700 45489


In [42]:
# підрахунок суми рядків
# вар 2
n_gate_30, n_gate_40 = len(gate_30_sample1), len(gate_40_sample1)

print(count_gate_30, count_gate_40, n_gate_30, n_gate_40)

44700 45489 44700 45489


In [43]:
# Конверсія (доля тих хто повернувся) в кожній групі вар1


p_gate_30 = retention_7_gate_30 / n_gate_30
p_gate_40 = retention_7_gate_40 / n_gate_40

print(p_gate_30, p_gate_40)

0.19020134228187918 0.18200004396667327


In [44]:
# Конверсія (доля тих хто повернувся) в кожній групі вар2
mean_retention_7_gate_30 = df[df['version'] == 'gate_30']['retention_7'].mean()
mean_retention_7_gate_40 = df[df['version'] == 'gate_40']['retention_7'].mean()

print(mean_retention_7_gate_30, mean_retention_7_gate_40)

0.19020134228187918 0.18200004396667327


In [46]:
# Обєднана конверсія
p_combined = (retention_7_gate_30 + retention_7_gate_40) / (n_gate_30 + n_gate_40)

# Стандартне відхилення обєднаної конверсії
se_combined = np.sqrt(p_combined * (1 - p_combined) * (1/n_gate_30 + 1/n_gate_40))

# Z-статистика
z_stat = (p_gate_30 - p_gate_40) / se_combined

# p-значення для двухстороннього тесту
p_value = 2 * (1 - stats.norm.cdf(np.abs(z_stat)))

print(f"Z-статистика для утримання гравця через 7 днів: {z_stat}")
print(f"P-значення для утримання гравця через 7 днів: {p_value}")

Z-статистика для утримання гравця через 7 днів: 3.164358912748191
P-значення для утримання гравця через 7 днів: 0.0015542499756142636


In [48]:
#Висновок:
if(p_value <  alpha):
    print("Відхилити Н0, дані не мають нормальний розподіл. Тобто переміщення воріт на 10 рівнів вплине на утримання гравців, що встановили гру 7 днів тому")
else:
    print("Н0 не може бути відхилена, дані мають нормальний розподіл.")

Відхилити Н0, дані не мають нормальний розподіл. Тобто переміщення воріт на 10 рівнів вплине на утримання гравців, що встановили гру 7 днів тому


2. Перевірте з допомогою z-тесту аналогічно до прикладу в лекції, чи дає якась з версій гри кращий показник `retention_7` на рівні значущості 0.05. Обчисліть також довірчі інтервали для двох вибірок. Виведіть результат у форматі:
```
z statistic: ...
p-value: ...
Довірчий інтервал 95% для групи control: [..., ...]
Довірчий інтервал 95% для групи treatment: [..., ...]
```
де замість `...` - обчислені значення. В якості висновка дайте відповідь на два питання:  
    1. чи є статистична значущою різниця між поведінкою користувачів у різних версіях гри?   
    2. чи перетинаються довірчі інтервали утримання користувачів з різних версій гри? Про що це каже?  
    
Зверніть увагу, в такому і схожому завданнях ми використовуєм `proportion` Z-тест. Це тому що в нас залежна змінна має бінарне значення (повернеться аби ні користувач, чи клікне або ні користувач в інших ситуаціях - всього два можливих значення в змінної: 0/1, True/False ). Якщо б ми вимірювали скажімо чи є стат. значущою різниця між вагою чоловіків і жінок в певній вибірці, ми б використовувавли функцію `statsmodels.stats.ztest`, бо залежна змінна `вага` є неперервною (тип float, замість типу int чи bool і тільки двох можливих значень).

In [51]:
# Доверительный интервал для группы gate_30
ci_30 = stats.norm.interval(0.95, loc=p_gate_30, scale=np.sqrt(p_gate_30 * (1 - p_gate_30) / n_gate_30))

# Доверительный интервал для группы gate_40
ci_40 = stats.norm.interval(0.95, loc=p_gate_40, scale=np.sqrt(p_gate_40 * (1 - p_gate_40) / n_gate_40))

print(f"z statistic для утримання гравця через 7 днів: {z_stat}")
print(f"p-value для утримання гравця через 7 днів: {p_value}")
print(f"Довірчий інтервал для групи gate_30: {ci_30}")
print(f"Довірчий інтервал для групи gate_40: {ci_40}")

z statistic для утримання гравця через 7 днів: 3.164358912748191
p-value для утримання гравця через 7 днів: 0.0015542499756142636
Довірчий інтервал для групи gate_30: (0.18656311652199903, 0.19383956804175934)
Довірчий інтервал для групи gate_40: (0.17845430073314686, 0.18554578720019968)


**Інтерпретація:**
**Перекриття інтервалів:**

Довірчі інтервали для двох груп не перекриваються. Верхня межа довірчого інтервалу для gate_40 (0.1855) нижче за нижню межу довірчого інтервалу для gate_30 (0.1866). Це означає, що різниця між групами статистично значуща лише на рівні довіри 95%.
**Середні значення конверсії:**

Середнє значення конверсії групи gate_30 лежить в інтервалі від 18.66% до 19.38%.
Середнє значення конверсії групи gate_40 лежить в інтервалі від 17.85% до 18.55%.
**Висновки про вплив зміни:**

Оскільки довірчі інтервали не перекриваються, можна з високою ймовірністю стверджувати, що переміщення воріт рівня 30 на рівень 40 дійсно вплинуло на конверсію. Група gate_30 показує вищі значення утримання через 7 днів порівняно

з gate_40.
**Додаткові висновки**:
**Практична значимість**:
Різниця у відсотках може бути відносно невеликою, але якщо гра має велику кількість користувачів, навіть невелика зміна утримання може суттєво впливати на загальні показники.
**Подальший аналіз:**

Варто також розглянути та проаналізувати утримання через 7 днів, а також інші метрики, щоб отримати повніше уявлення про вплив зміни рівня воріт.
Можуть бути інші фактори, що впливають на утримання, які слід врахувати, наприклад, складність рівнів, внутрішньоігрові події і таке інше.
Таким чином, довірчі інтервали показують статистично значущу різницю між групами, що дозволяє зробити висновок про вплив переміщення воріт на утримання гравців.

3. Є ще один тип тестів, який використовується для бінарної метрики як от "зробить юзер дію, чи ні" - тест [**Хі-квадрат**](https://www.bmj.com/about-bmj/resources-readers/publications/statistics-square-one/8-chi-squared-tests) (ще одне [пояснення](https://www.scribbr.com/statistics/chi-square-tests/) тесту з прикладами). В нього інші гіпотези Н0 і Н1 на відміну від z- та t-тестів. А також цей тест можна використовувати, якщо в нас більше за 2 досліджувані групи, тобто в нас не А/В тест, а А/B/C/D, наприклад.  

В **z- та t-тестах** (які відрізняються тим, що ми в першому не знаємо дисперсію генеральної сукупності, але якщо в нас великий набір даних, то ці два тести дають дуже схожі результати) **ми перевіряємо, чи є різниця у середніх показниках по групам користувачів**.  

А в **тесті Хі-квадрат ми перевіряємо чи є звʼязок між групою користувача і тим, чи він зробить цікаву нам дію**. Це ніби дослідження одного і того самого, але дещо різними способами. Для перевірки, можна виконувати кілька тестів (особливо, якщо один дає якийсь непереконливий результат типу р-значення 0.07 - наче і fail to regect H0 на рівні стат значущості 5%, але цікаво, що скажуть інші тести), тож, зробимо і ми тест хі-квадрат та порівняємо його результат з z-тестом.

Про різницю між тестами можна почитати ще [тут](https://stats.stackexchange.com/a/178860) - це просто пояснення користувача стековерфлоу, але там розумні люди сидять.

Для проведення хі-квадрат тесту скористаємось функцією з `scipy.stats` `chi2_contingency` для обчислення статистики хі-квадрат і р-значення для перевірки конкретної гіпотези. У цю функцію вам треба передати таблицю 2х2: кількість випадків для кожної версії гри і значення `retention_7`.

**Задача**: виконайте тест хі-квадрат на рівні значущості 5% аби визначити, чи є залежність між версією гри та тим, чи зайде гравець на 7ий день після встановлення гри.
Тут гіпотези наступні
- Н0: значення retention_7 не залежить від версії гри
- Н1: є залежність між версією гри і значенням retention_7

Виведіть p-значення та зробіть висновок.


In [52]:
from scipy import stats

In [53]:
# Создаем таблицу сопряженности
contingency_table = pd.crosstab(df['version'], df['retention_7'])

print(contingency_table)

retention_7  False  True 
version                  
gate_30      36198   8502
gate_40      37210   8279


In [54]:
# Выполняем тест хи-квадрат
chi2, p, dof, expected = stats.chi2_contingency(contingency_table)

print(f"Хи-квадрат статистика: {chi2}")
print(f"P-значение: {p}")

Хи-квадрат статистика: 9.959086799559167
P-значение: 0.0016005742679058301


In [55]:
# Проверяем значение p-value для уровня значимости 0.05
alpha = 0.05
if p < alpha:
    print("Отклоняем нулевую гипотезу. Есть зависимость между версией игры и значением retention_7.")
else:
    print("Не отклоняем нулевую гипотезу. Нет зависимости между версией игры и значением retention_7.")

Отклоняем нулевую гипотезу. Есть зависимость между версией игры и значением retention_7.
